In [5]:
import pydicom
import tempfile
import datetime
from pydicom.dataset import Dataset, FileDataset

def generate_dicom_from_pdf(pdf_file):
    suffix = '.dcm'
    filename = tempfile.NamedTemporaryFile(suffix=suffix).name

    file_meta = Dataset()
    file_meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.104.1'
    file_meta.MediaStorageSOPInstanceUID = '2.16.840.1.114430.287196081618142314176776725491661159509.60.1'
    file_meta.ImplementationClassUID = '1.3.46.670589.50.1.8.0'
    file_meta.TransferSyntaxUID = '1.2.840.10008.1.2.1'

    ds = FileDataset(filename, {},
                 file_meta=file_meta, preamble=b"\0" * 128)

    ds.is_little_endian = True
    ds.is_implicit_VR = False

    dt = datetime.datetime.now()
    ds.ContentDate = dt.strftime('%Y%m%d')
    timeStr = dt.strftime('%H%M%S.%f')
    ds.ContentTime = timeStr

    ds.SOPClassUID = '1.2.840.10008.5.1.4.1.1.104.1'

    with open(pdf_file, 'rb') as f:
        f_read = f.read()
        ValueLength = len(f_read)
        ## All Dicom Element must have an even ValueLength
        if ValueLength % 2 != 0:
            f_read += b'\0'
        ds.EncapsulatedDocument = f_read

    ds.MIMETypeOfEncapsulatedDocument = 'application/pdf'

    ds.Modality = 'DOC' #secondary capture
    ds.ConversionType = 'WSD' #workstation
    ds.SpecificCharacterSet = 'ISO_IR 100' 
    # more codes for charecter encoding here https://dicom.innolitics.com/ciods/cr-image/sop-common/00080005 

    return ds

In [6]:
import argparse
import datetime

def generate_random_uid():
    return '1.9.9.' + str(datetime.datetime.now().strftime('%H%M%S%f%d%m%Y'))

def main(input_pdf):

    e_pdf_ds = generate_dicom_from_pdf(input_pdf)

    e_pdf_ds.PatientName = 'John Doe'
    e_pdf_ds.PatientID = '12345'
    e_pdf_ds.PatientSex = 'M'
    e_pdf_ds.StudyInstanceUID = generate_random_uid()

    e_pdf_ds.SeriesInstanceUID = generate_random_uid()
    e_pdf_ds.SOPInstanceUID = generate_random_uid() 

    e_pdf_ds.save_as('/Users/andreasala/Desktop/pippo.dcm')

In [7]:
main('/Users/andreasala/Desktop/pippo.pdf')

In [21]:
from pdf2dcm import Pdf2EncapsDCM

converter = Pdf2EncapsDCM()
converted_dcm = converter.run(path_pdf='/Users/andreasala/Desktop/dummy.pdf',
                     #path_template_dcm='/Users/andreasala/Desktop/Tesi/data/COVID-NOCOVID/2020019001_103984788_Torace3.0Bl571/CT/00240001.dcm',
                     suffix =".dcm")
print(converted_dcm)
# tests/test_data/test_file'

/Users/andreasala/Desktop/dummy.dcm


In [19]:
import pdf2dcm
from pdf2dcm.base import BaseConverter

converter = BaseConverter()
converted_dcm = converter.run(path_pdf='/Users/andreasala/Desktop/dummy.pdf',
                     #path_template_dcm='/Users/andreasala/Desktop/Tesi/data/COVID-NOCOVID/2020019001_103984788_Torace3.0Bl571/CT/00240001.dcm',
                     suffix =".dcm")
print(converted_dcm)

TypeError: Can't instantiate abstract class BaseConverter with abstract method run